## 训练系统更新 (2025-07-24)

新增功能:
- 早停机制: 验证准确率50步无提升自动停止
- 学习率调度: 验证准确率plateau时自动降低学习率
- 验证集支持: 自动使用validation_formatted.jsonl

技术实现:
- 早停: EarlyStopping回调，监控val_accuracy
- 学习率调度: ReduceLROnPlateau，学习率减半，最小lr=1e-7
- 数据分离: train/validation/test 三重分离

训练命令模板:
```bash
cd /root/PAW/train_lora && python train_cs_lora_lightning.py \
  --dataset arc-challenge \
  --base_model /root/autodl-tmp/models/Qwen_Qwen2.5-1.5B \
  --lr 1e-5 \
  --max_steps 2000 \
  --bs 6
```

注意: max_steps现在是上限，实际可能因早停而提前结束

In [ ]:
## 训练命令模板

基础训练:
```bash
cd /root/PAW/train_lora && python train_cs_lora_lightning.py \
  --dataset arc-challenge \
  --base_model /root/autodl-tmp/models/Qwen_Qwen2.5-1.5B \
  --lr 1e-5 \
  --max_steps 150 \
  --bs 4
```

参数说明:
- lr: 学习率，推荐1e-5到1.5e-5之间
- max_steps: 训练步数，600步左右效果较好
- bs: 批大小，4-6适中

In [ ]:
7.24 0:15 修好了全repo的test的一致性 
并且跑了2个baseline 和 分别微调(batch=4, step=600)的acc

In [ ]:
python eval/lightning_eval.py --models_list /root/PAW/runs/Qwen_Qwen2.5-1.5B/arc-challenge_lora_2025
0723_191421/final_model --dataset arc-challenge --base_model /root/autodl-tmp/model
s/Qwen_Qwen2.5-1.5B


lightning_eval.py 说明 如果是test trained model 要加base_model，这样才能识别是在什么基模之上的delta W.

perplexity 请忽略 没啥用

In [ ]:
基准测试结果 arc-challenge

Model                                    Dataset         Loss     Accuracy   Perplexity   
----------------------------------------------------------------------------------------
gemma-2-2b-it                            arc-challenge   34.9178  0.7491     17695200835
gemma-2-2b-it_lora_trained               arc-challenge   0.3224   0.7381     1.3813

Qwen_Qwen2.5-1.5B                        arc-challenge   9.6764   0.7338     16587.6250
Qwen_Qwen2.5-1.5B_20250723_191421        arc-challenge   0.2720   0.0597     1.3135    (失败训练)
Qwen_Qwen2.5-1.5B_20250724_005835        arc-challenge   0.2675   0.2193     1.3075    (失败训练)
Qwen_Qwen2.5-1.5B_20250724_013028        arc-challenge   0.1490   0.7346     1.1616    (150步)

说明: 
- gemma基础模型性能较好 0.7491，LoRA后略降至0.7381
- Qwen基础模型0.7338，经过参数调优后LoRA训练成功

In [ ]:
实验记录 - Qwen2.5-1.5B LoRA训练

基础模型准确率: 0.7338

训练结果对比:
步数150, lr=1e-5, bs=4:     0.7346 (+0.08%)
步数200, lr=1.5e-5, bs=6:   0.7346 (+0.08%)  
步数600, lr=1.5e-5, bs=6:   0.7457 (+1.19%)
步数600, lr=2e-5, bs=6:     0.7363 (+0.25%) 过高学习率反而下降
步数2000, lr=1e-5, bs=6:    0.7210 (-1.28%) 过拟合

关键发现:
1. 600步是较好的停止点，2000步开始过拟合
2. lr=1.5e-5在600步时效果最佳
3. lr=2e-5过激进，lr=1e-5在长步数下会过拟合
4. 早停和验证集机制已实现但此次未触发

最佳配置: lr=1.5e-5, max_steps=600, bs=6
最佳准确率: 0.7457 (提升1.19个百分点)

## 评估脚本参数说明

### 当前命令分析：
```bash
python eval/lightning_eval.py \
  --models_list /root/PAW/runs/gemma-2-2b-it/arc-challenge_lora_20250723_235207/final_model \
  --dataset arc-challenge \
  --base_model /root/autodl-tmp/models/gemma-2-2b-it \
  --sample_ratio 0.2
```

### 参数含义：
- `--models_list`: **要评估的模型路径**（可以是LoRA模型或基础模型）
- `--base_model`: **仅当评估LoRA模型时需要**，用于加载基础模型再应用LoRA权重
- `--dataset`: 数据集名称
- `--sample_ratio`: 采样比例

### 如何评估原始基础模型：
```bash
# 评估未训练的原始 Gemma 模型
python eval/lightning_eval.py \
  --models_list /root/autodl-tmp/models/gemma-2-2b-it \
  --dataset arc-challenge
  # 注意：不需要 --base_model 参数！
```